In [2]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='sypark', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: sypark...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:26


## Day 7 

In [13]:
pcs5 = load_data_mart("pc", start_date = "2021-09-08", end_date = "2021-09-14", table_name = "gcoin_use")

In [14]:
pcs6 = load_data_mart("pc", start_date = "2022-03-16", end_date = "2022-03-22", table_name = "gcoin_use")

In [15]:
pcs5_au = load_data_mart('pc', start_date = "2021-09-14", end_date = "2021-09-14", table_name = "user_master").where(col('lastlogindate') >= "2021-09-08")

In [16]:
pcs6_au = load_data_mart('pc', start_date = "2022-03-22", end_date = "2022-03-22", table_name = "user_master").where(col('lastlogindate') >= "2022-03-16")

In [17]:
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

In [18]:
pcs5_au = pcs5_au.withColumn("country_new", country_type_udf("country_os", "country_ip"))
pcs5_au = pcs5_au.join(meta_region, pcs5_au.country_new == meta_region.country_code_iso2, "left")

pcs6_au = pcs6_au.withColumn("country_new", country_type_udf("country_os", "country_ip"))
pcs6_au = pcs6_au.join(meta_region, pcs6_au.country_new == meta_region.country_code_iso2, "left")

In [19]:
pcs5 = pcs5.where(lower(col("event_name")).like("%pcs5%"))
pcs6 = pcs6.where(lower(col("event_name")).like("%pcs6%"))

In [20]:
pcs5_df = pcs5.join(pcs5_au, pcs5.account_id == pcs5_au.accountid, "left")
pcs6_df = pcs6.join(pcs6_au, pcs6.account_id == pcs6_au.accountid, "left")

In [25]:
pcs5_df.groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), (sum(col("paid_use"))/100).alias("paid_gcoin"), countDistinct(when(col("paid_use") > 0, col("account_id"))).alias("paid_pu")).toPandas()

,pubg_region,pu,paid_gcoin,paid_pu
0,CN,86816,1210928.9,73775
1,NA,1331,23811.4,1275
2,SA,1209,14926.5,1048
3,KR,22202,380190.2,20324
4,SEA,5746,74230.3,4960
5,JP,382,7928.1,368
6,TW/HK,833,13831.1,750
7,CIS,1964,23617.8,1732
8,EMEA,4109,68109.4,3863
9,OC,492,8405.1,459


In [26]:
pcs6_df.groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), (sum(col("paid_use"))/100).alias("paid_gcoin"), countDistinct(when(col("paid_use") > 0, col("account_id"))).alias("paid_pu")).toPandas()

,pubg_region,pu,paid_gcoin,paid_pu
0,CN,36036,470342.35,25338
1,NA,1395,17313.20,992
2,None,1,0.00,0
3,SA,1234,8338.10,594
4,KR,13169,162245.30,9140
5,SEA,2115,18467.90,1208
6,JP,193,2685.30,137
7,TW/HK,459,6660.30,327
8,CIS,2780,8623.91,756
9,EMEA,4702,44307.80,2758


In [29]:
pcs5_au.groupBy("pubg_region").agg(countDistinct("accountid")).toPandas()

,pubg_region,count(DISTINCT accountid)
0,CN,2194430
1,NA,75680
2,SA,58320
3,KR,638133
4,SEA,136405
5,Undefined,41
6,JP,12335
7,TW/HK,27110
8,CIS,122399
9,EMEA,216200


In [30]:
pcs6_au.groupBy("pubg_region").agg(countDistinct("accountid")).toPandas()

,pubg_region,count(DISTINCT accountid)
0,CN,2873541
1,NA,133180
2,SA,133901
3,KR,687346
4,SEA,225468
5,Undefined,793
6,JP,17899
7,TW/HK,38182
8,CIS,381787
9,EMEA,441999


PCS5 Paid PU

In [31]:
#pcs5 day7
pcs5_df.select(countDistinct(when(col("paid_use") > 0, col("account_id")))).show()

+------------------------------------------------------------+
|count(DISTINCT CASE WHEN (paid_use > 0) THEN account_id END)|
+------------------------------------------------------------+
|                                                      108554|
+------------------------------------------------------------+



In [32]:
#pcs5 day1
load_data_mart("pc", start_date = "2021-09-08", end_date = "2021-09-08", table_name = "gcoin_use").where(lower(col("event_name")).like("%pcs5%")).select(countDistinct(when(col("paid_use") > 0, col("account_id")))).show()

+------------------------------------------------------------+
|count(DISTINCT CASE WHEN (paid_use > 0) THEN account_id END)|
+------------------------------------------------------------+
|                                                       45513|
+------------------------------------------------------------+



In [33]:
#pcs5 day3
load_data_mart("pc", start_date = "2021-09-08", end_date = "2021-09-10", table_name = "gcoin_use").where(lower(col("event_name")).like("%pcs5%")).select(countDistinct(when(col("paid_use") > 0, col("account_id")))).show()

+------------------------------------------------------------+
|count(DISTINCT CASE WHEN (paid_use > 0) THEN account_id END)|
+------------------------------------------------------------+
|                                                       78920|
+------------------------------------------------------------+

